# Generating of cylindric mesh

In [15]:
import numpy
from dolfin import *
from mshr import *

Theta = 1 * pi
a, b = 0.1, 3.0
nr = 10  # divisions in r direction
nt = 20  # divisions in theta direction
mesh = UnitSquareMesh(10, 10, 'crossed')
#mesh = generate_mesh(Rectangle(Point(0, 0), Point(1, 1)), 10)
#mesh = RectangleMesh(nr, nt, digonal='crossed')

# First make a denser mesh towards r=a
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]
s = 1

def denser(x, y):
    return [a + (b-a)*((x-a)/(b-a))**s, y]

x_bar, y_bar = denser(x, y)
xy_bar_coor = numpy.array([x_bar, y_bar]).transpose()
mesh.coordinates()[:] = xy_bar_coor
plot(mesh, title='stretched mesh')

def cylinder(r, s):
    return [r*numpy.cos(Theta*s), r*numpy.sin(Theta*s)]

x_hat, y_hat = cylinder(x_bar, y_bar)
xy_hat_coor = numpy.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
plot(mesh, title='hollow cylinder')
interactive()

# Solution of infinite string problem

## "Circle - Circle" approach

In [7]:
from dolfin import *
from mshr import *

r, R = 0.1, 3.
count = 30

mesh = generate_mesh(Circle(Point(0., 0.), R, 30) - Circle(Point(0., 0.), r, 20), count)

#plot(mesh)

#'''
def outer_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] > r + 0.1)

def inner_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] < r + 0.1)
#'''

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

#'''
bcs = [DirichletBC(V, Constant(1.), outer_boundary),
       DirichletBC(V, Constant(10.), inner_boundary)]
#'''

#bcs = DirichletBC(V, Expression('1/sqrt(x[0]*x[0]+x[1]*x[1])'), lambda x, on_boundary: on_boundary)

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)

#E = -project(grad(u), VectorFunctionSpace(mesh, "Lagrange", 1))

#plot(E)

interactive()

In [2]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri

triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                           triangles=mesh.cells())
Z = u.compute_vertex_values(mesh)

plt.figure()
plt.tricontour(triang, Z)
plt.colorbar()
plt.show()

## PointSource approach

In [32]:
from dolfin import *
from mshr import *

R = 3.
count = 30

mesh = generate_mesh(Circle(Point(0., 0.), R, 30), count)

plot(mesh)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

bc = DirichletBC(V, Constant(1 / R), lambda x, on_boundary: on_boundary)

A, b = assemble_system(a, L, bc)

delta = PointSource(V, Point(0., 0.,), 1.)
delta.apply(b)

u = Function(V)
solve(A, u.vector(), b)

plot(u)

u_e = Expression('1 / sqrt(x[0]*x[0] + x[1]*x[1])')

plot(u - u_e)

interactive()

# Solution of infinite plate problem

## "Circle - Rectangle" + "Inner Values" approach

In [4]:
from dolfin import *
from mshr import *


def solve_problem(U, l, w, infinity, mode='boundary'):
    if mode == 'slice':
        mesh = generate_mesh(Circle(Point(0., 0.), infinity, 30) - Rectangle(Point(-l, -w), Point(l, w)), 50)
        def outer_boundary(x, on_boundary):
            return on_boundary and not inner_boundary(x, on_boundary)
        def inner_boundary(x, on_boundary):
            return on_boundary and near(x[1], 0., eps=w) and near(x[0], 0., eps=l)
    else:
        mesh = generate_mesh(Circle(Point(0., 0.), 3, 30), 50)
        def outer_boundary(x, on_boundary):
            return on_boundary
        def inner_boundary(x, on_boundary):
            return near(x[1], 0., eps=0.05) and -1 <= x[0] <= 1

    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [DirichletBC(V, Constant(0 * ln(1 / infinity)), outer_boundary),
           DirichletBC(V, Constant(U), inner_boundary)]

    a = inner(grad(u), grad(v)) * dx
    L = Constant(0.0) * v * dx

    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()

    plot(u)
    #plot(mesh)
    interactive()
    
    return u, mesh

In [5]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri

def make_contour_plot(u, mesh, levels):
    triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                               triangles=mesh.cells())
    Z = u.compute_vertex_values(mesh)

    plt.figure()
    plt.tricontour(triang, Z, levels)
    plt.colorbar()
    return plt

In [6]:
u, mesh = solve_problem(100., 1., 0.001, 10., mode='slice')
plt = make_contour_plot(u, mesh, 17)
plt.savefig('contour_plot.png')

# Задача для кругового цилиндирического бесконечного конденсатора

In [10]:
from dolfin import *
from mshr import *

r, R = 0.1, 3.
U1, U2 = 1., 10.
count = 30

mesh = generate_mesh(Circle(Point(0., 0.), R, 30) - Circle(Point(0., 0.), r, 20), count)

def outer_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] > r + 0.1)

def inner_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] < r + 0.1)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(U1), outer_boundary),
       DirichletBC(V, Constant(U2), inner_boundary)]

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

u_e = Expression("(U2 - U1)/log(r/R) * log(x[0]*x[0]+x[1]*x[1])/2", U1=U1, U2=U2, r=r, R=R)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)
plot(u_e, mesh=mesh)
plot(u - u_e)

#E = -project(grad(u), VectorFunctionSpace(mesh, "Lagrange", 1))

#plot(E)

interactive()

# Density

In [30]:
from dolfin import *
from mshr import *

R = 10.
w = 0.01
l = 3.
count = 10
tol = 10E-14

mesh = generate_mesh(Rectangle(Point(0, -R), Point(R, R)) - Rectangle(Point(0, -w), Point(l, w)), count)

'''
class LeftDoundary(SubDomain):
    def left_boundary(x, on_boundary):
        return on_boundary and near(x[0], 0.)
'''

class OuterBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and (near(abs(x[0]), R) or near(abs(x[1]), R))

class InnerBoundary(SubDomain):
    def inside(self, x, on_boundary):
        if on_boundary and x[0] < l - tol and abs(x[1]) < w + tol:
            print 'OK'
            return True
        return False

boundary_parts = MeshFunction('uint', mesh, mesh.topology().dim() - 1)

Gamma_N = InnerBoundary()
Gamma_N.mark(boundary_parts, 0)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Expression('-log(x[0]*x[0]+x[1]*x[1])/2.'), OuterBoundary())]

#e = Constant(100.)
e = Expression('sin(x[0])')

a = inner(grad(u), grad(v)) * dx
L = e * v * ds(0)

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)
plot(mesh)

#E = -project(grad(u), VectorFunctionSpace(mesh, "Lagrange", 1))

#plot(E)

interactive()


levels = 100


import matplotlib.pyplot as plt
import matplotlib.tri as tri

triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                           triangles=mesh.cells())
Z = u.compute_vertex_values(mesh)

plt.figure()
plt.tricontour(triang, Z, levels)
plt.colorbar()
plt.show()

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
